# Disk Health Predictor Usage Demo

In this notebook, we will show how you can get started with the [`disk-health-predictor`](https://github.com/aicoe-aiops/disk-health-predictor) python module with just a few lines of code! First we will install it as a package, and then we will pass to it a [`smartctl`](https://linux.die.net/man/8/smartctl) json collected from a real hard disk, to predict that hard disk's health and estimate its time-to-failure.

## Installation

To install Disk Health Predictor using pip, you can run the following command:
```bash
pip install git+https://github.com/aicoe-aiops/disk-health-predictor.git
```
Alternatively, you can also add it to your Pipfile like this:
```
[[source]]
name = "pypi"
url = "https://pypi.org/simple"
verify_ssl = true

[dev-packages]
some-packege = "*"

[packages]
disk-health-predictor = {git = "https://github.com/aicoe-aiops/disk-health-predictor.git", ref="master"}
another-pkg = "*"

[requires]
python_version = "3.8"
```

In [1]:
# lets install using the command line for now
# !pip install git+https://github.com/aicoe-aiops/disk-health-predictor.git

## Usage

To collect your disk's SMART data using the `smartctl` tool, you can run the following command (replacing `/dev/nvme0` with the device you want)
```bash
smartctl --json --xall /dev/nvme0
```

To collect and share (anonymously) this data for the hard disks in your Ceph cluster, you can enable the [Telemetry](https://docs.ceph.com/en/latest/mgr/telemetry/) module. We will use one of the smartctl jsons collected via Ceph telemetry as sample input data in this notebook.

In [2]:
import json
from pprint import pprint

In [3]:
# lets take a look at what the input data looks like
with open("sample_input_1.json", "r") as f:
    sample_smartctl_jsons = json.load(f)

# view keys
keys = sample_smartctl_jsons.keys()
print("Keys in the input JSON:")
print(keys, end="\n\n")

# view value for a key
sample_key = list(keys)[0]
print(f"Value for Key {sample_key} in the input JSON:")
pprint(sample_smartctl_jsons[sample_key], depth=4)

Keys in the input JSON:
dict_keys(['2020-04-01 00:08:56', '2020-04-02 00:00:59', '2020-04-02 09:38:43', '2020-04-03 00:03:17', '2020-04-04 00:05:22', '2020-04-05 00:07:31', '2020-04-06 00:09:38', '2020-04-07 00:00:20'])

Value for Key 2020-04-01 00:08:56 in the input JSON:
{'ata_sct_capabilities': {'data_table_supported': True,
                          'error_recovery_control_supported': True,
                          'feature_control_supported': True,
                          'value': 28861},
 'ata_smart_attributes': {'revision': 10,
                          'table': [{'flags': {...},
                                     'id': 1,
                                     'name': 'Raw_Read_Error_Rate',
                                     'raw': {...},
                                     'thresh': 44,
                                     'value': 80,
                                     'when_failed': '',
                                     'worst': 64},
                              

As we can see, the keys in the input json are the datetime objects when the `smartctl` output was taken, and values are the raw `smartctl` json output. Note that predictor needs at least 6 days of data to make a valid prediction.

Disk Health Predictor provides several `Classifier`s, such `PSDiskHealthClassifier` and `RHDiskHealthClassifier`, to estimate disk health using `smartctl` data. These classifiers make use of some pretrained models associated with them to predict a disk's health as either
- `BAD` (will fail in less than 2 weeks),
- `WARNING` (will fail in 2 to 6 weeks),
- `GOOD` (will not fail for at least 6 weeks), or
- `UNKNOWN` (could not generate a prediction due to insufficient data).

To instantiate these classifiers, you can use the factory function `DiskHealthClassifierFactory` as shown below.

In [4]:
from disk_health_predictor.classifiers import DiskHealthClassifierFactory  # noqa: E402

In [5]:
clf = DiskHealthClassifierFactory("redhat")
clf

Now, you can classify disk health to estimate when the disk will fail as follows.

In [6]:
pred = clf.predict(sample_smartctl_jsons)
print(f"This hard disk is estimated to be in the '{pred}' health category.")

This hard disk is estimated to be in the 'Good' health category.


/home/kachau/.local/share/virtualenvs/demo-dhp-U8zZfpVL/lib/python3.8/site-packages/sklearn/base.py:301: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/kachau/.local/share/virtualenvs/demo-dhp-U8zZfpVL/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/kachau/.local/share/virtualenvs/demo-dhp-U8zZfpVL/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `

If you have any questions, comments, or suggestions, please feel free to [open issues](https://github.com/aicoe-aiops/disk-health-predictor/issues/new/choose) or [submit pull requests](https://github.com/aicoe-aiops/disk-health-predictor/compare)!